# Datenbankschema (DDL)

In [2]:
USE master;
CREATE DATABASE m106
GO

: Msg 1801, Level 16, State 3, Line 2
Database 'm106' already exists. Choose a different database name.

Total execution time: 00:00:00.001

In [3]:
USE m106;
GO
CREATE SCHEMA verkauf;
GO

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 3
There is already an object named 'verkauf' in the database.

: Msg 2759, Level 16, State 0, Line 3
CREATE SCHEMA failed due to previous errors.

Total execution time: 00:00:00.003

In [4]:
-- Tabelle kunden im Schema verkauf erstellen
CREATE TABLE verkauf.kunden (
    kundenid INT IDENTITY(1,1),
    vorname VARCHAR(30),
    nachname VARCHAR(30)
);
-- Lege zum Vergleich dieselbe Tabelle im Standardschema an

-- Tabelle bestellungen im Schema verkauf erstellen
CREATE TABLE verkauf.bestellungen (
    bestellid INT IDENTITY(1,1),
    bestelldatum DATE,
    kundenid INT
);

Commands completed successfully.

Total execution time: 00:00:00.019

In [1]:
-- Aufruf der gespeicherten Prozedur sp_columns
EXEC sp_columns @table_name = N'kunden',  @table_owner = N'verkauf'; 

(0 rows affected)

Total execution time: 00:00:00.365

TABLE_QUALIFIER,TABLE_OWNER,TABLE_NAME,COLUMN_NAME,DATA_TYPE,TYPE_NAME,PRECISION,LENGTH,SCALE,RADIX,NULLABLE,REMARKS,COLUMN_DEF,SQL_DATA_TYPE,SQL_DATETIME_SUB,CHAR_OCTET_LENGTH,ORDINAL_POSITION,IS_NULLABLE,SS_DATA_TYPE


In [3]:
-- neue Spalte bestellwert hinzufügen
USE m106
ALTER TABLE verkauf.bestellungen ADD bestellwert DECIMAL(10,2);

Commands completed successfully.

Total execution time: 00:00:00.008

In [4]:
EXEC sp_rename 'verkauf.bestellungen.bestellwert', 'bestellbetrag', 'COLUMN';

Caution: Changing any part of an object name could break scripts and stored procedures.

Total execution time: 00:00:00.520

In [5]:
-- Länge der Spalte vornamen auf 50 ändern
ALTER TABLE verkauf.kunden ALTER COLUMN vorname VARCHAR(50);

Commands completed successfully.

Total execution time: 00:00:00.006

In [6]:
-- Datentyp der Spalte bestelldatum zu DATETIME ändern
ALTER TABLE verkauf.bestellungen ALTER COLUMN bestelldatum DATETIME;

Commands completed successfully.

Total execution time: 00:00:00.009

In [7]:
-- Spalte bestellbetrag aus der Tabelle löschen
ALTER TABLE verkauf.bestellungen DROP column bestellbetrag;

Commands completed successfully.

Total execution time: 00:00:00.009

In [8]:
-- Tabelle bestellungen im Schema verkauf löschen
DROP TABLE verkauf.bestellungen;

Commands completed successfully.

Total execution time: 00:00:00.018

In [9]:
CREATE TABLE verkauf.bestellungen(
    bestellid INT IDENTITY(1,1),
    bestelldatum DATETIME,
    kundenid INT
);

Commands completed successfully.

Total execution time: 00:00:00.008

In [10]:
-- Gesamten Inhalt der Tabelle bestellungen löschen
TRUNCATE TABLE verkauf.bestellungen;

Commands completed successfully.

Total execution time: 00:00:00.002

In [11]:
-- Primärschlüssel beim Erstellen der neuen Tabelle produkte definieren
CREATE TABLE verkauf.produkte (
    produktid INT IDENTITY(1,1) PRIMARY KEY,
    produktname VARCHAR(100),
    preis MONEY,
    energielabel CHAR(1)
);

Commands completed successfully.

Total execution time: 00:00:00.010

In [12]:
-- Primärschlüssel auf der Tabelle kunden nachträglich definieren
ALTER TABLE verkauf.kunden ADD PRIMARY KEY (kundenid);

-- Primärschlüssel auf der Tabelle bestellungen nachträglich definieren
ALTER TABLE verkauf.bestellungen ADD PRIMARY KEY (bestellid);

Commands completed successfully.

Total execution time: 00:00:00.011

In [13]:
/*
Fremdschlüssel auf der Tabelle bestellungen definieren, welcher auf die Primär-
schlüsselspalte der Tabelle kunden verweist
*/
ALTER TABLE verkauf.bestellungen  
ADD CONSTRAINT fk_bestellungen_kunden FOREIGN KEY (kundenid) 
REFERENCES verkauf.kunden(kundenid);

/* 
Identisches Beispiel mit dem Unterschied, dass der Constraint nicht explizit 
benannt wird

ALTER TABLE verkauf.bestellungen  
ADD FOREIGN KEY (kundenid) REFERENCES verkauf.kunden(kundenid);
*/

Commands completed successfully.

Total execution time: 00:00:00.006

In [14]:
-- keine leeren Werte in der Spalte nachname zulassen
ALTER TABLE verkauf.kunden ALTER COLUMN nachname VARCHAR(30) NOT NULL;

Commands completed successfully.

Total execution time: 00:00:00.006

In [15]:
-- Standartwert in der Spalte vorname setzen
ALTER TABLE verkauf.kunden ADD DEFAULT 'unbekannt' FOR vorname;

Commands completed successfully.

Total execution time: 00:00:00.010

In [16]:
/* 
Regel hinzufügen, dass in der Spalte bestelldatum nur Datumswerte eingefügt werden
dürfen, welche nicht älter als 01.01.2020 sind
*/
ALTER TABLE verkauf.bestellungen ADD CHECK (bestelldatum > '2019-12-31');

-- Regel definieren, dass in der Spalte energielabel nur bestimmte Werte erlaubt sind
ALTER TABLE verkauf.produkte ADD CHECK (energielabel IN ('A','B','C','D','E'));

Commands completed successfully.

Total execution time: 00:00:00.012

In [ ]:
USE master;
GO

ALTER DATABASE m106 SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
GO
DROP DATABASE m106;
GO